In [1]:
from brainlit.utils import session, upload_benchmarking
from pathlib import Path
import napari

c:\users\shrey\anaconda3\envs\ndd-windows\lib\site-packages\python_jsonschema_objects\__init__.py:53: UserWarning: Schema version http://json-schema.org/draft-04/schema not recognized. Some keywords and features may not be supported.
  self.schema["$schema"]


## Uploading Benchmarking Images from local data locations .
This notebook demonstrates uploading the benchmarking data and associated `.swc` segment files.
The upload destination could easily be set to a url of a cloud data server such as s3.

## 1) Define variables.
 - `source` and `source_segments` are the root directories of the octree-formatted data and swc files.
 - `p` is the prefix string. `file://` indicates a filepath, while `s3://` or `gc://` indicate URLs.
 - `dest` and `dest_segments` are the destinations for the uploads (in this case, filepaths).
 - `num_res` denotes the number of resolutions to upload. If uploading .tif files that aren't very large or where only one resolution is necessary, set num_res to 1. 
 
The below paths lead to sample data in the NDD Repo. Alter the below path definitions to point to your own local file locations.

## 2) Upload the image data (.tif)

In [ ]:
upload_benchmarking.upload_volumes(source, dest, num_res)

If the upload fails with the error: `timed out on a chunk on layer index 0. moving on to the next step of pipeline`, re-run the `upload_volumes` function but with the `continue_upload` parameter, which takes `layer index` (the layer index said in the error message) and `image index` (the last succesful image that uploaded).  

For example, if the output failed after image 19, then run 
`upload_benchmarking.upload_volumes(source, dest, num_res, continue_upload = (0, 19))`. Repeat this till all of the upload is complete.


## 3) Upload the segmentation data (.swc)

In [ ]:
upload_benchmarking.upload_segments(source, dest_segments, num_res)